# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [48]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama
import whisper

In [47]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 73.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803406 sha256=00b8314a9075ced02e7e61196cc7c513cbedecba7d515a65f967685696ada3d9
  Stored in directory: /Users/alexh/Library/Caches/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper
Note: you may need to restart the kernel to use updated packages.


In [38]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()
MODEL_LLAMA = 'llama3.2'

OpenAI API Key exists and begins sk-proj-


In [6]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

In [25]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [26]:
flight_data = {
    "london": {
        "flights": [
            {"flight_number": "BA117", "departure": "2025-04-21T10:00", "arrival": "2025-04-21T14:00", "available_seats": 5},
            {"flight_number": "BA119", "departure": "2025-04-21T16:00", "arrival": "2025-04-21T20:00", "available_seats": 3}
        ]
    },
    "paris": {
        "flights": [
            {"flight_number": "AF123", "departure": "2025-04-22T09:00", "arrival": "2025-04-22T13:00", "available_seats": 4},
            {"flight_number": "AF125", "departure": "2025-04-22T15:00", "arrival": "2025-04-22T19:00", "available_seats": 2}
        ]
    },
    "tokyo": {
        "flights": [
            {"flight_number": "JL101", "departure": "2025-04-23T08:00", "arrival": "2025-04-23T16:00", "available_seats": 6},
            {"flight_number": "NH202", "departure": "2025-04-23T12:00", "arrival": "2025-04-23T20:00", "available_seats": 4},
            {"flight_number": "JL303", "departure": "2025-04-23T18:00", "arrival": "2025-04-24T02:00", "available_seats": 5}
        ]
    },
    "berlin": {
        "flights": [
            {"flight_number": "LH789", "departure": "2025-04-24T07:00", "arrival": "2025-04-24T11:00", "available_seats": 3},
            {"flight_number": "FR456", "departure": "2025-04-24T13:00", "arrival": "2025-04-24T17:00", "available_seats": 2},
            {"flight_number": "EZ123", "departure": "2025-04-24T19:00", "arrival": "2025-04-24T23:00", "available_seats": 4}
        ]
    }
    # Add more destinations as needed
}

In [27]:
def get_booking_details(destination_city):
    print(f"Tool get_booking_details called for {destination_city}")
    city = destination_city.lower()
    return flight_data.get(city, {"flights": []})

In [28]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [29]:
booking_function = {
    "name": "get_booking_details",
    "description": "Retrieve available flight options for a given destination, including flight times and seat availability. Use this when a customer wants to know about flight schedules or availability.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the customer wishes to travel to."
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [30]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

## Getting OpenAI to use our Tool


In [31]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        messages.append(assistant_message)

        # Process each tool call
        for tool_call in assistant_message.tool_calls:
            tool_response = handle_tool_call(tool_call)
            messages.append(tool_response)

        # Get the final response after tool execution
        final_response = openai.chat.completions.create(model=MODEL, messages=messages)
        return final_response.choices[0].message.content

    return response.choices[0].message.content

In [32]:
def handle_tool_call(tool_call):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")

    if function_name == "get_ticket_price":
        result = get_ticket_price(city)
        content = json.dumps({"destination_city": city, "price": result})
    elif function_name == "get_booking_details":
        result = get_booking_details(city)
        content = json.dumps(result)
    else:
        content = json.dumps({"error": f"Function {function_name} not implemented."})

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": content
    }

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [15]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [16]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

# For Windows users (or any Mac users with problems above)

## First try the Mac version above, but if you get a permissions error writing to a temp file, then this code should work instead.

A collaboration between students Mark M. and Patrick H. and Claude got this resolved!

Below are 4 variations - hopefully one of them will work on your PC. If not, message me please!

And for Mac people - all 3 of the below work on my Mac too - please try these if the Mac version gave you problems.

## PC Variation 1

In [20]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

# Our Agent Framework


In [65]:
def translate_to_romanian(text):
    messages = [
        {"role": "system", "content": "You are a translation assistant."},
        {"role": "user", "content": f"Translate the following text to Romanian:\n\n{text}"}
    ]
    response = ollama.chat(model=MODEL_LLAMA, messages=messages)
    return response['message']['content']

In [66]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        messages.append(assistant_message)

        for tool_call in assistant_message.tool_calls:
            tool_response = handle_tool_call(tool_call)
            messages.append(tool_response)

            # If the tool is get_ticket_price or get_booking_details, generate an image
            if tool_call.function.name in ["get_ticket_price", "get_booking_details"]:
                arguments = json.loads(tool_call.function.arguments)
                city = arguments.get("destination_city")
                image = artist(city)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]

    # Translate the reply to Romanian
    translation = translate_to_romanian(reply)

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)

    return history, image, translation

In [67]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
        translation_output = gr.Textbox(label="Romanian Translation", lines=10)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output, translation_output]
    )
    clear.click(lambda: None, inputs=None, outputs=[chatbot, translation_output], queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7932

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin


Tool get_ticket_price called for Tokyo
